In [ ]:
%load_ext kamu

In [ ]:
import kamu
con = kamu.connect(engine="spark", connection_params=dict(driver_memory="1000m", executor_memory="2000m"))

# Land value heatmap
Let's join land tax report records to their corresponding geographical boundaries and visualize their price on a map.

In [ ]:
%%sql
select * from `ca.vancouver.opendata.property.tax-reports` limit 3

In [ ]:
%%sql
select * from `ca.vancouver.opendata.property.parcel-polygons` limit 3

In [ ]:
%%sql -o heatmap -q
select
    tax.report_year,
    tax.pid,
    tax.legal_type,
    tax.zoning_district,
    cast(tax.current_land_value as double) + cast(tax.current_improvement_value as double) as current_total_value,
    polys.geometry
from `ca.vancouver.opendata.property.parcel-polygons` as polys
inner join `ca.vancouver.opendata.property.tax-reports` as tax
    on tax.land_coordinate = polys.tax_coord
where
    tax.legal_type = 'LAND'
    and tax.tax_assessment_year = 2024
    and tax.current_land_value is not null

In [ ]:
import kamu.utils

heatmap_gj = kamu.utils.df_to_geojson(heatmap)

In [ ]:
import os
import kamu.utils
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

viz = ChoroplethViz(
    heatmap_gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=10,
    access_token=token,
    color_property='current_total_value',
    color_stops=create_color_stops([1000000, 2000000, 3000000, 5000000, 10000000], colors='YlOrRd'),
    color_default='rgb(158,202,195)',
    line_width=0,
    opacity=1.0,
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)

viz.show()

# Spatial JOIN
We have two GIS datasets with outlines of every city block and geo boundaries of city neighbourhoods. Let's classify which neighbourhood every city block belongs to by joining two datasets using `st_contains` to test that a block polygon is fully contained within a neighbourhood polygon.

In [ ]:
%%sql
select * from `ca.vancouver.opendata.property.block-outlines` limit 3

In [ ]:
%%sql
select * from `ca.vancouver.opendata.property.local-area-boundaries` limit 3

In [ ]:
%%sql -o blocks_by_hood -q
with blocks as (
    select
        st_geomfromgeojson(geometry) as geometry
    from `ca.vancouver.opendata.property.block-outlines`
),
hoods as (
    select
        st_geomfromgeojson(geometry) as geometry,
        name
    from `ca.vancouver.opendata.property.local-area-boundaries`
),
blocks_by_hood as (
    select hoods.name, blocks.geometry
    from
        blocks,
        hoods
    where st_intersects(blocks.geometry, hoods.geometry)
)
select
    st_asgeojson(geometry) as geometry,
    name,
    -- calculating median value is left as an excercise :)
    rand() as median_value
from blocks_by_hood

In [ ]:
import kamu.utils

blocks_by_hood_gj = kamu.utils.df_to_geojson(blocks_by_hood)

In [ ]:
viz = ChoroplethViz(
    blocks_by_hood_gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=10,
    access_token=token,
    color_property='median_value',
    color_stops=create_color_stops([0.25, 0.5, 0.75, 1.0], colors='YlOrRd'),
    line_stroke='solid',
    line_width=0.1,
    line_color='rgb(128,0,38)',
    opacity=0.8,
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)

viz.show()

In [ ]:
# adjust view angle
viz.bearing = -15
viz.pitch = 45

# add extrusion to viz using interpolation keyed on density in GeoJSON features
viz.height_property = 'median_value'
viz.height_stops = create_numeric_stops([0, 1], 0, 500)
viz.height_function_type = 'interpolate'
viz.opacity = 1

# render again
viz.show()